# Data prep:  Content Based filtering using audio features, album_uri and artist_uri

In [1]:
import import_ipynb

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import scipy.sparse as sps

from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from category_encoders import TargetEncoder
from time import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#own functions
from evaluation import DCG
from evaluation import nDCG
from evaluation import R_Precision


%matplotlib inline

# Data transformation, PCA and merging 

In [ ]:
with open('../data-processed/full-data/track_descriptions.json') as json_file:
    D_desc = json.load(json_file)
    
D_desc['spotify:track:0UaMYEvWZi0ZqiDOoHU3YI']

In [ ]:
with open('../data-processed/full-data/track_artist_album.json') as json_file:
    D_album_artist = json.load(json_file)
    
D_album_artist['spotify:track:0UaMYEvWZi0ZqiDOoHU3YI']

In [ ]:
path = '../data-processed/full-data/audio-features-combined.csv'
data = pd.read_csv(path)

In [ ]:
data = data.reset_index()
data = data.rename(columns = {'index':'track_id'})
data['artist_uri'] = data.apply(lambda x:D_album_artist[x.uri][0], axis = 1)
data['album_uri'] = data.apply(lambda x:D_album_artist[x.uri][1], axis = 1)

In [ ]:
# data.head()

In [ ]:
D_track_uri_to_id = data.groupby('uri')['track_id'].min().to_dict()
D_track_id_to_uri = data.groupby('track_id')['uri'].min().to_dict()

In [ ]:
len(D_track_uri_to_id), len(D_track_id_to_uri)

In [ ]:
data.head()

In [ ]:
##
genres = pd.read_csv('../data-processed/full-data/genres_by_artist.csv')

In [ ]:
##
genres.head()

In [ ]:
##
##data_genres = data.merge(genres, how='left', left_on='artist_uri', right_on='artist_uri')

In [ ]:
# data_genres.shape

# PCA on genres

In [ ]:
X_genres = genres.iloc[:,1:]

In [ ]:
X_genres.shape

In [ ]:
X_genres.values

In [ ]:
pca = PCA(n_components=9000)
pca = pca.fit(X_genres.values)
print(sum(pca.explained_variance_ratio_))

In [ ]:
X_genres_pca = pca.transform(X_genres)

In [ ]:
genres_pca_df = pd.DataFrame(X_genres_pca)

In [ ]:
genres_pca_df.shape

In [ ]:
genres_pca_df.shape, genres.shape

In [ ]:
genres_pca_df['artist_uri'] = genres['artist_uri']

In [ ]:
new_cols = ['pca_genre_'+ str(el) for el in list(genres_pca_df.columns)]

In [ ]:
genres_pca_df.columns = new_cols

In [ ]:
genres_pca_df.head()

In [ ]:
genres_pca_df.to_csv('../data-processed/transformation-matrices/cb_genres_pca_df.csv', index = None)

# Plot data

In [ ]:
# data.iloc[:,1:-1].hist(figsize=(14,9))
# uncomment
# plt.show()

In [ ]:
# fig, ax = plt.subplots(ncols=4, nrows=3, figsize=(14,9))

# column = data.columns[2:14]

#uncomment
# for i in range(3):
#     for j in range(4):
#         sns.boxplot(data=data[column[i*4+j]], palette="Set1", ax=ax[i,j]).set_title(column[i*4+j])

# Standardize data 

In [ ]:
X = data.iloc[:,2:13]

In [ ]:
#column orders
data.iloc[:,2:13].columns

In [ ]:
scaler = StandardScaler()
transformer = scaler.fit(X)
X_transformed = transformer.transform(X)

In [ ]:
# uncomment
# pd.DataFrame(X_transformed).hist(figsize=(14,9))
# plt.show()

# Target encode album_uri and artist_uri

In [ ]:
X_transformed.shape

In [ ]:
df_X_transformed = pd.DataFrame(X_transformed, columns=data.iloc[:,2:13].columns)
# df_X_transformed.head()

In [ ]:
df_X_transformed.shape

In [ ]:
df_X_transformed.columns

# Encode album_uri

In [ ]:
cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

In [ ]:
df_X_transformed['album_uri'] = data.album_uri
X = data['album_uri'].values.reshape(-1,1)

In [ ]:
enc = TargetEncoder()
for col in cols:
    y = df_X_transformed[col].values.reshape(-1,1)  
    df_X_transformed[f'album_uri_{col}_enc'] = enc.fit_transform(X, y)

In [ ]:
# df_X_transformed.head()

# Encode artist_uri

In [ ]:
cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

In [ ]:
df_X_transformed['artist_uri'] = data.artist_uri
X = data['artist_uri'].values.reshape(-1,1)

In [ ]:
enc = TargetEncoder()
for col in cols:
    y = df_X_transformed[col].values.reshape(-1,1) 
    df_X_transformed[f'artist_uri_{col}_enc'] = enc.fit_transform(X, y)

In [ ]:
# df_X_transformed.head()

In [ ]:
#X_transformed = df_X_transformed.drop(columns=['artist_uri','album_uri']).to_numpy()

In [ ]:
df_X_transformed.shape

In [ ]:
df_X_transformed.head()

In [ ]:
df_X_transformed.to_csv('../data-processed/transformation-matrices/cb_df_X_transformed.csv', index = None)

# Add Genres PCA

In [ ]:
df_X_transformed.columns

In [ ]:
df_X_transformed

In [ ]:
df_merged = df_X_transformed.merge(genres_pca_df, how = 'left', left_on = 'artist_uri', right_on ='pca_genre_artist_uri' )

In [ ]:
df_merged.shape

In [ ]:
df_merged.head()

In [ ]:
#df_merged = df_merged.fillna(0)

In [ ]:
df_merged.to_csv('../data-processed/transformation-matrices/cb_df_merged.csv', index = None)

In [ ]:
X_transformed = df_merged.drop(columns=['artist_uri','album_uri']).to_numpy()